In [1]:
import pandas as pd
from datetime import date
import requests
from bs4 import BeautifulSoup

In [2]:
source=requests.get("https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations").text
soup=BeautifulSoup(source,'lxml')
table=soup.find_all('table')
rows = soup.find_all('tr')[12:68]
states_abbreviations={}
for row in rows:
    cells = row.find_all('td')
    states_abbreviations.update({cells[3].text:cells[0].text.strip()})

In [3]:
us_infections=pd.read_csv(r"covid19/infections_timeseries.csv")


us_infections['state'] = us_infections.apply(lambda row: str(row.Combined_Key).split('-')[1].strip(' '), axis = 1)
FIPS=us_infections[["FIPS","Combined_Key","state"]].set_index("FIPS")

us_infections.drop(columns=["FIPS"])
infections_by_city = dict(tuple(us_infections.groupby('state')))
infections_by_state = us_infections.groupby('state').sum()

In [4]:
us_deaths=pd.read_csv(r"covid19/deaths_timeseries.csv").drop(columns=["FIPS"])

us_deaths['state'] = us_deaths.apply(lambda row: str(row.Combined_Key).split('-')[1].strip(' '), axis = 1)
deaths_by_city = dict(tuple(us_deaths.groupby('state')))
deaths_by_state = us_deaths.groupby('state').sum()

In [5]:
us_interventions=pd.read_csv(r"covid19/interventions.csv").drop(columns=["FIPS"],index=[0])

for i in list(us_interventions.columns)[2:]:
    us_interventions[i] = us_interventions[i].dropna().apply(int).apply(date.fromordinal)
us_interventions['STATE'] = us_interventions.apply(lambda row: states_abbreviations[row.STATE] , axis = 1)
interventions_by_state = us_interventions.groupby('STATE').first().drop(columns=["AREA_NAME"])

In [6]:
us_poi_visits=pd.read_csv(r"covid19/poi_visits.csv").set_index("FIPS")

us_poi_visits['state'] = None
us_poi_visits['Combined_Key'] = None
for i in us_poi_visits.index:
    try:us_poi_visits.at[i,'state']=FIPS.at[i,"state"]
    except:None
    try:us_poi_visits.at[i,'Combined_Key']=FIPS.at[i,'Combined_Key']
    except:None
poi_visits_by_state = us_poi_visits.groupby('state').sum()

In [7]:
datasets=[states_abbreviations,FIPS,infections_by_city,infections_by_state,deaths_by_city,
          deaths_by_state,us_interventions,interventions_by_state,us_poi_visits,poi_visits_by_state]